In [59]:
?make_classification

In [1]:
import torch
import numpy as np

#### Pre-Requisites:
- Python
- Numpy

[YT Playlist link: PyTorch 101: An Applied Tutorial](https://www.youtube.com/playlist?list=PL98nY_tJQXZln8spB5uTZdKN08mYGkOf2)

In [2]:
torch.cuda.is_available()

False

In [3]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


## TENSORS

In [4]:
#array
some_data =[[1,2],[3,4]]
type(some_data)

list

In [7]:
some_data=torch.tensor(some_data) #or as np.asarray(some_data)
print(type(some_data))
print(some_data.dtype)

<class 'torch.Tensor'>
torch.int64


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [9]:
torch.from_numpy(np.asarray(some_data))
torch.tensor(np.array(some_data))

tensor([[1, 2],
        [3, 4]])

In [11]:
ta=torch.zeros(3,4) #Similarly, ones/ random values.

In [15]:
ta.device #Not torch.device(), it's for fixing the cpu/gpu
# ta.to("cuda") ot ta.to("cuda:0") to specify the gpu number

device(type='cpu')

In [16]:
tax = ta.to("cuda")

AssertionError: Torch not compiled with CUDA enabled

In [17]:
ta.device

device(type='cpu')

- We can do indexing, slicing, element-wise multiplication, transpose, matrix multiplication (using "@") like numpy
- Also, functions like sum, max, min, shape, size, clip(low, high) using <code>tensor.x()</code> functions like numpy
- element wise multiplying using:<br>
<code>tensor_array1.mult(tensor_array2)</code>
- matrix multiplication using:<br>
<code>tensor_array1.matmult(tensor_array2)</code><br>
<code>torch.matmult(tensor_array1, tensor_array2)</code>
- Concatenation:<br>
<code>torch.cat([tensor_array1, tensor_array2],dim=1)</code>
<br><br>
- "Obviously, we have to take care of the dimensions."

In [22]:
#examples function as:
ta = np.random.rand(3,4)
ta = torch.tensor(ta)
print(ta)
torch.nn.functional.softmax(ta, dim=0)

tensor([[0.3249, 0.1609, 0.8677, 0.8268],
        [0.1478, 0.1249, 0.7671, 0.9436],
        [0.9078, 0.9605, 0.1838, 0.5243]], dtype=torch.float64)


tensor([[0.2756, 0.2387, 0.4151, 0.3493],
        [0.2308, 0.2303, 0.3754, 0.3926],
        [0.4936, 0.5310, 0.2095, 0.2581]], dtype=torch.float64)

In [24]:
#Random images
torch.rand(10, 3, 128, 128).size()
# batch size at 1st location
# number of channels at 2nd location
# image size at next 2 location(s)

torch.Size([10, 3, 128, 128])

## AUTOGRAD

In [43]:
a = torch.tensor([5.], requires_grad = True)
b = torch.tensor([6.], requires_grad = True)

In [44]:
y= a**3 - b**2

In [45]:
# Let error be like this:
y
#  And we know:
# doy/doa = 3*(a**2) = 75
# doy/dob = -2*b = -12

tensor([89.], grad_fn=<SubBackward0>)

In [46]:
print(a.grad)
y.backward() # maybe to calculate gradients using auto-grad
print(a.grad)
print(b.grad)

None
tensor([75.])
tensor([-12.])


To check:<br>
(Why output like this after three repetion of running the codes)<br>
tensor([150.])<br>
tensor([225.])<br>
tensor([-36.])<br>

In [51]:
w = torch.rand(10,1, requires_grad=True)
b = torch.rand(1, requires_grad=True)
x = torch.rand(1,10)
out = x@w + b
loss= 1-out

print(w,b,x,out,loss)

tensor([[0.7095],
        [0.7509],
        [0.5247],
        [0.4116],
        [0.3638],
        [0.0960],
        [0.5623],
        [0.6655],
        [0.8262],
        [0.3311]], requires_grad=True) tensor([0.9106], requires_grad=True) tensor([[0.3519, 0.0395, 0.4577, 0.9362, 0.7947, 0.4333, 0.5006, 0.7190, 0.0061,
         0.8038]]) tensor([[3.1772]], grad_fn=<AddBackward0>) tensor([[-2.1772]], grad_fn=<RsubBackward1>)


In [53]:
loss.backward()

In [54]:
w.grad

tensor([[-0.3519],
        [-0.0395],
        [-0.4577],
        [-0.9362],
        [-0.7947],
        [-0.4333],
        [-0.5006],
        [-0.7190],
        [-0.0061],
        [-0.8038]])

In [55]:
with torch.no_grad():
    w = w - 0.001*w.grad.data

print(w)

tensor([[0.7098],
        [0.7510],
        [0.5252],
        [0.4125],
        [0.3646],
        [0.0964],
        [0.5628],
        [0.6662],
        [0.8262],
        [0.3319]])


## DATASET CLASS

In [7]:
# class CustomData(torch.utils.data.Dataset):
class CustomData:
    def __init__(self, data, targets):
        self.data=data
        self.targets=targets
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        current_s = self.data[idx, :]
        current_t = self.targets[idx]
        return {
            "sample":torch.tensor(current_s, dtype=torch.float),
            "target":torch.tensor(current_t, dtype=torch.long)
        }
    
# Generally, people prefer "x" for "sample" and "y" for "target".
# dtype depends on the loss function and other properties too.

In [8]:
from sklearn.datasets import make_classification

In [9]:
data, target = make_classification(n_samples=1000)
print(data.shape)
print(target.shape)

(1000, 20)
(1000,)


In [62]:
custom_dataset =CustomData(data = data, targets=target)

In [64]:
print(len(custom_dataset[0]))
print(len(custom_dataset))
print(custom_dataset[0])

2
1000
{'sample': tensor([-0.7619, -1.9836,  1.2194,  0.5783,  0.7072,  0.7332,  1.0867,  0.2879,
        -0.9986, -0.3439,  0.2289, -1.9558,  0.8709,  0.1011,  1.2127,  0.2182,
         1.2840, -1.4816,  0.9693,  1.2223]), 'target': tensor(1)}


In [65]:
# Application:

for idx in range(len(custom_dataset)):
    print(custom_dataset[idx])
    if idx>5:
        break

{'sample': tensor([-0.7619, -1.9836,  1.2194,  0.5783,  0.7072,  0.7332,  1.0867,  0.2879,
        -0.9986, -0.3439,  0.2289, -1.9558,  0.8709,  0.1011,  1.2127,  0.2182,
         1.2840, -1.4816,  0.9693,  1.2223]), 'target': tensor(1)}
{'sample': tensor([-1.7080, -0.4159, -1.4802, -0.0622, -0.7584, -1.5648,  0.5263,  0.6660,
         1.2169,  0.6066, -0.2656, -0.8406, -0.3472, -2.3983,  0.1130,  1.6105,
        -0.3942, -0.2746, -0.2896, -0.0594]), 'target': tensor(1)}
{'sample': tensor([-1.1130, -1.1842,  0.7222, -0.0590,  0.4818,  0.6935,  0.8553, -1.0046,
         0.9994, -0.9671, -0.8262, -0.4588, -1.5403,  1.1588,  0.6262,  0.1429,
         0.2467,  0.9598,  1.3079,  0.5113]), 'target': tensor(1)}
{'sample': tensor([-0.0190, -0.6118, -0.0570, -0.8735, -0.6569,  0.4228, -0.8071, -1.4655,
        -0.0970, -1.2110, -0.1714,  0.1229, -1.3995,  0.5562,  0.9148, -0.3354,
         2.0152, -1.1634, -1.1972,  1.5851]), 'target': tensor(0)}
{'sample': tensor([ 2.1890,  1.2367,  1.7232, -2

## EXAMPLE: DATASET CLASS FOR TEXT/NLP PROBLEM

In [67]:
# Classification/Regression Problem
class CustomData:
    def __init__(self, data, targets, tokenizer):
        self.data=data
        self.targets=targets
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]
        # There could more than one texts to be maintained.
        # Thus, similarly tests will be reported and return.... 
        # .....in the following dictionary
        target = self.targets[idx] 
        # Considering labels are not multi-labeled.
        # else: target = self.targets[idx,:]
        # Example: for entitiy classification problem
        
        input_ids = tokenizer(text)
        # padding should be taken care for
        # it could return attention_mask(?)
        
        return {
            "text":torch.tensor(input_ids, dtype=torch.long),
            "target":torch.tensor(target, dtype=torch.long)
            # "attention_mask":torch.tensor(attention_mask, dtype=torch.long)
            # if tokenizer returns attention_mask too
        }
    
# We need tokenizer for converting text into set of number(s) or related.
# Generally, people prefer "x" for "sample" and "y" for "target".
# dtype depends on the loss function and other properties too.

## EXAMPLE: DATASET CLASS FOR IMAGE/VISION PROBLEM

In [69]:
import cv2

# Image Classification/Segmentation Problem
class CustomData:
    def __init__(self, image_paths, targets, augmnetation=None):
        self.image_paths=image_paths
        self.targets=targets
        self.augmnetation = augmnetation
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = cv2.imread(self.image_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        target = self.targets[idx] 
        
        if self.augmnetation is not None:
            augmentedfn = self.augmnetation(image =  image)
#             augmentedfn = self.augmnetation[image=image, mask=mask]
#             mask = augmentedfn["mask"]
# mask has to be read explicitly and can be returned in the dictionary output
            image = augmentedfn["image"]
        # implementations-> library will be useful to do the 
        # changes rather than doing it yourself
        # maybe tried same through this if thing(?) 
        
        # tensor.unsqueeze(0)
        # for grayscale image, it has 1 channel only
        
        # tensor expect images as channel-first type, 
        # hence we have to do transpose
        image = image.transpose(2,0,1).astype(np.float32)
        
        return {
            "image":torch.tensor(image),
            "target":torch.tensor(target)
        }
    
# Generally, people prefer "x" for "sample" and "y" for "target".
# dtype depends on the loss function and other properties too.

## DATALOADER

In [10]:
#Using CustomData class object as of Lecture 3
dataset = CustomData(data, target)

In [11]:
len(dataset)

1000

In [13]:
?torch.utils.data.DataLoader

In [14]:
train_loader =  torch.utils.data.DataLoader(dataset, batch_size=4, num_workers=2)

In [15]:
for data in train_loader:
    print(data)
    print(data["sample"].shape)
    print(data["target"].shape)
    break

{'sample': tensor([[-0.8891, -0.5090,  0.6909, -0.5496,  0.5080, -0.8045,  0.5756,  0.6849,
         -1.3066, -2.3758,  0.5251, -1.2246,  0.6970,  0.3190,  1.6159,  0.3006,
          1.3427, -1.1055, -0.4909,  1.1992],
        [-1.7564,  0.4141, -0.5333,  1.0336,  0.1155, -1.6251,  0.3973, -1.5293,
          0.8759, -0.2205, -1.3711, -1.0370, -1.9178, -0.7788,  1.1603,  1.3506,
         -0.2443, -0.4613,  0.1826,  0.3824],
        [ 0.1111,  0.2983,  0.0936,  0.2896,  0.2179, -0.4096, -1.1693,  0.5330,
         -0.5110,  1.1761,  0.7897,  0.9047,  0.8359,  0.2775,  1.9568, -1.7094,
         -2.9740, -2.0807, -1.1070,  1.1738],
        [ 0.4920, -0.5056, -0.2491, -0.1135, -0.0666,  1.6544,  0.0458,  1.2709,
         -1.8455,  1.0235, -0.8137, -1.3915, -1.7636, -0.4477, -0.0799, -0.6705,
          0.8853, -0.4362,  0.3929, -0.4600]]), 'target': tensor([0, 1, 0, 0])}
torch.Size([4, 20])
torch.Size([4])


In [16]:
for _ in range(10): #Epochs
    for data in train_loader:
        x =data["sample"]
        y = data["target"]
#       -> output = model(x,y) # Forward-Pass
        #loss
        #loss.backward()
        #optimizer
        #.....(and other steps)